# Cleaning Workzone Data from 2018-2024
#### Import Libraries

In [19]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd

#### Read data and clean it

In [20]:
data = pd.read_excel("CMU RCRS Data - 2017-18/Events Report 2018 - 2020.xlsx")

# Preliminarily how many work zones per year in the raw file?
data['BEGINNING_DATE'] = pd.to_datetime(data['BEGINNING_DATE'])
data['Year'] = data['BEGINNING_DATE'].dt.year
data.groupby('Year').size()


Year
2018    69320
2019    74850
2020    64483
dtype: int64

In [21]:
wdata=data.copy()
wdata.head()

,EVENTID,DISTRICT,STATE_ROUTE,DIRECTION,FROM_LOCATION,END_LOCATION,INCIDENT_LOCATION,INFO_PROVIDED_BY,VERIFIED,CAUSE,...,IS_DETOUR_IN_PLACE,IS_DETOUR_EFFECTIVE,DETOUR_DESCRIPTION,FIRST_RESPONDERS_ARRIVE,PENNDOT_ARRIVE,DURATION_IN_HOURS,BEGIN_LOCATION_LATLONG,END_LOCATION_LATLONG,INCIDENT_LOCATION_LATLONG,Year
0,357901,District 08,2018 (CHOCOLATE AV / WALTON AV / BRIDGE RD / ...,BOTH,HUMMELSTOWN [BORO]_!_ROSANNA ST DAUPHIN (22),HUMMELSTOWN [BORO]_!_HANOVER ST DAUPHIN (22),,DEAN MARRIOTT,01/26/2018 18:06:00 OTHER,SPECIAL EVENT,...,T,T,NaN,NaN,NaN,4.776667,"40.2652210679873,-76.7098815896065","40.2653242756258,-76.707868762623",NaN,2018
1,357920,District 08,0114 (PA - 114),BOTH,0262 (PA - 262) - FISHING CREEK RD in FAIRVIE...,FAIRVIEW [2TWP]_!_RUDYTOWN RD YORK (66),,RUDY HUGGINS,01/09/2018 08:45:00 PENNDOT PERSONNEL,ROADWORK,...,T,T,NaN,NaN,NaN,557.599722,"40.1815337221234,-76.8788784102723","40.1884307467765,-76.8893313120578",NaN,2018
2,359196,District 08,4001 (OLD CARLISLE RD),BOTH,BUTLER [2TWP]_!_GUERNSEY RD ADAMS (01),BUTLER [2TWP]_!_CENTER MILLS RD ADAMS (01),,RODNEY GIBSON / CLEARWATER CONST,01/02/2018 08:20:00 PRIVATE CONTRACTOR,BRIDGE OUTAGE,...,F,NaN,NaN,NaN,NaN,5354.028056,"39.953891008477,-77.211833076343","39.9553087688289,-77.2112441196716",NaN,2018
3,360002,District 08,3047 (IRON RIDGE RD / MAPLE GROVE RD / KRAFTS...,BOTH,0516 (PA - 516) - BERLIN STREET in JEFFERSON ...,NORTH CODORUS [2TWP]_!_PARK RD YORK (66),,CLINT LEAKWAY,01/09/2018 09:01:00 PENNDOT PERSONNEL,ROADWORK,...,F,NaN,NaN,NaN,NaN,748.562222,"39.8210804126407,-76.8451685782727","39.8180782732112,-76.8726839582193",NaN,2018
4,361537,District 06,0095 (I - 095),SOUTH,GIRARD AVE/LEHIGH AVE { # 23 } PHILADELPHIA (67),ALLEGHENY AVE/CASTER AVE { # 25 } PHILADELPHI...,,CARLTON MCVEIGH,12/31/2017 23:59:00 CCTV,DISABLED VEHICLE,...,F,NaN,NaN,NaN,NaN,0.492778,"39.983192889809,-75.0986559582808","39.9781083768692,-75.1113919811173",NaN,2018


In [22]:
wdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208653 entries, 0 to 208652
Data columns (total 42 columns):
 #   Column                                            Non-Null Count   Dtype         
---  ------                                            --------------   -----         
 0   EVENTID                                           208653 non-null  int64         
 1   DISTRICT                                          208653 non-null  object        
 2   STATE_ROUTE                                       208653 non-null  object        
 3   DIRECTION                                         208653 non-null  object        
 4   FROM_LOCATION                                     208653 non-null  object        
 5   END_LOCATION                                      208653 non-null  object        
 6   INCIDENT_LOCATION                                 208653 non-null  object        
 7   INFO_PROVIDED_BY                                  208653 non-null  object        
 8   VERIFIED      

In [23]:
# Drop unnecessary columns
wdata.drop(columns=['CLOSURE_INFORMATION', 'IS_DETOUR_EFFECTIVE', 'DETOUR_DESCRIPTION', 'FIRST_RESPONDERS_ARRIVE', 'PENNDOT_ARRIVE'], inplace=True)

In [24]:
# Filter based on clean data
wdata = wdata[(wdata['END_LOCATION_LATLONG'].notnull()) & (wdata['BEGIN_LOCATION_LATLONG'].notnull()) & \
    (wdata['END_LOCATION_LATLONG'] != ",") & (wdata['BEGIN_LOCATION_LATLONG'] != ",")]
wdata.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149087 entries, 0 to 208652
Data columns (total 37 columns):
 #   Column                                            Non-Null Count   Dtype         
---  ------                                            --------------   -----         
 0   EVENTID                                           149087 non-null  int64         
 1   DISTRICT                                          149087 non-null  object        
 2   STATE_ROUTE                                       149087 non-null  object        
 3   DIRECTION                                         149087 non-null  object        
 4   FROM_LOCATION                                     149087 non-null  object        
 5   END_LOCATION                                      149087 non-null  object        
 6   INCIDENT_LOCATION                                 149087 non-null  object        
 7   INFO_PROVIDED_BY                                  149087 non-null  object        
 8   VERIFIED           

In [25]:
# Convert to date time
wdata['ACT_DATETIME_OPENED'] = pd.to_datetime(wdata['ACT_DATETIME_OPENED'])
wdata['Year'] = wdata['ACT_DATETIME_OPENED'].dt.year

In [26]:
# Filter by cause the duration
wdata = wdata[wdata['CAUSE'] == 'ROADWORK']
wdata = wdata[(wdata['DURATION_IN_HOURS'] <= (7*24)) & (wdata['DURATION_IN_HOURS'] > 0)]

In [27]:
# How many are left per year?
data.groupby('Year').size()

Year
2018    69320
2019    74850
2020    64483
dtype: int64

In [28]:
wdata = wdata[['EVENTID', 'DIRECTION', 'STATE_ROUTE', 'BEGIN_LOCATION_LATLONG', 'END_LOCATION_LATLONG', 'BEGINNING_DATE', \
    'ACT_DATETIME_OPENED', 'DURATION_IN_HOURS']]

In [29]:
# Clean st rt no and direction columns
wdata['ST_RT_NO'] = (wdata['STATE_ROUTE'].str[:4]).astype(int)
wdata.drop(columns=['STATE_ROUTE'], inplace=True)
wdata['DIRECTION'] = wdata['DIRECTION'].str[0]
wdata.head()

,EVENTID,DIRECTION,BEGIN_LOCATION_LATLONG,END_LOCATION_LATLONG,BEGINNING_DATE,ACT_DATETIME_OPENED,DURATION_IN_HOURS,ST_RT_NO
83,361616,E,"39.9528201289115,-76.7484358158499","39.9514275303733,-76.7476956127466",2018-01-02 07:17:50,2018-01-02 15:55:00,8.619444,3054
88,361621,B,"40.697267282843,-80.3890556951048","40.6942457919881,-80.373508844171",2018-01-02 08:16:00,2018-01-02 17:16:00,9.000000,4028
92,361625,B,"40.0471409885973,-75.7160158119286","40.035105842437,-75.7093626228477",2018-01-02 08:33:01,2018-01-02 08:34:00,0.016389,282
95,361628,S,"40.153599046815,-77.3023022732082","40.1293449089626,-77.3493401405175",2018-01-02 08:59:10,2018-01-02 14:50:00,5.847222,81
96,361629,B,"39.8376674032663,-75.6973863879682","39.8401605943598,-75.7100399702765",2018-01-02 09:07:04,2018-01-02 15:48:00,6.682222,82


In [30]:
# Write to file
wdata.to_csv("cleaned_2018-2020_workzones.csv")

In [31]:
# Again for the 2021-2024 file
data2 = pd.read_excel("CMU RCRS Data - 2017-18/Events Report 2021 - 2024.xlsx")
wdata2 = data2.copy()
wdata2.head()

,EVENTID,DISTRICT,STATE_ROUTE,DIRECTION,FROM_LOCATION,END_LOCATION,INCIDENT_LOCATION,INFO_PROVIDED_BY,VERIFIED,CAUSE,...,DESCRIPTION_OF_EVENT,IS_DETOUR_IN_PLACE,IS_DETOUR_EFFECTIVE,DETOUR_DESCRIPTION,FIRST_RESPONDERS_ARRIVE,PENNDOT_ARRIVE,DURATION_IN_HOURS,BEGIN_LOCATION_LATLONG,END_LOCATION_LATLONG,INCIDENT_LOCATION_LATLONG
0,570322,District 08,2080 (GRACETON RD),BOTH,"FAWN [2TWP]_!_BRYANSVILLE RD, CONSTITUTION RD...",PEACH BOTTOM [2TWP]_!_GROSS RD YORK (66),,ADAM HOFF,01/06/2021 07:46:00 PENNDOT PERSONNEL,ROADWORK,...,Planned Event Description History:\n\t01/06/20...,F,NaN,NaN,NaN,NaN,6.612222,"39.7212324915759,-76.4148035083266","39.7212224130221,-76.3964464770387",NaN
1,571023,District 08,4022 (LEWISBERRY RD / SIDDONSBURG RD),BOTH,FAIRVIEW [2TWP]_!_MARIE AV YORK (66),FAIRVIEW [2TWP]_!_WEST ST YORK (66),,KEITH DRIESBAUGH,01/14/2021 07:31:00 PENNDOT PERSONNEL,ROADWORK,...,Extend to 2/26,F,NaN,NaN,NaN,NaN,987.797222,"40.1391058074007,-76.8712487050507","40.1371588034077,-76.8668174250536",NaN
2,571083,District 09,0099 (I - 099),NORTH,(Beginning of I - 099) BEDFORD (05),(End of I - 099) CENTRE (14),,AREA COMMAND,01/01/2021 09:58:00,WINTER WEATHER,...,NaN,F,F,NaN,NaN,NaN,9.165000,"40.0558545550851,-78.5179413563517","40.9421749880512,-77.7256600364977",NaN
3,571084,District 09,0099 (I - 099),SOUTH,(End of I - 099) BEDFORD (05),(Beginning of I - 099) CENTRE (14),,AREA COMMAND,01/01/2021 09:59:00,WINTER WEATHER,...,NaN,F,F,NaN,NaN,NaN,9.160556,"40.9421749880512,-77.7256600364977","40.0558574483092,-78.5182656870055",NaN
4,571085,District 09,0099 (I - 099),NORTH,(Beginning of I - 099) BEDFORD (05),(End of I - 099) CENTRE (14),,AREA COMMAND,01/01/2021 10:03:00,WINTER WEATHER,...,NaN,F,F,NaN,NaN,NaN,9.024722,"40.0558545550851,-78.5179413563517","40.9421749880512,-77.7256600364977",NaN


In [32]:
data2['BEGINNING_DATE'] = pd.to_datetime(data2['BEGINNING_DATE'])
data2['Year'] = data2['BEGINNING_DATE'].dt.year
data2.groupby('Year').size()

Year
2021    71228
2022    72167
2023    77931
2024    35919
dtype: int64

In [33]:
wdata2['BEGINNING_DATE'] = pd.to_datetime(wdata2['BEGINNING_DATE'])
wdata2['Year'] = wdata2['BEGINNING_DATE'].dt.year
wdata2.groupby('Year').size()

Year
2021    71228
2022    72167
2023    77931
2024    35919
dtype: int64

In [34]:
wdata2 = wdata2[(wdata2['END_LOCATION_LATLONG'].notnull()) & (wdata2['BEGIN_LOCATION_LATLONG'].notnull()) & \
    (wdata2['END_LOCATION_LATLONG'] != ",") & (wdata2['BEGIN_LOCATION_LATLONG'] != ",")]
wdata2 = wdata2[wdata2['CAUSE'] == 'ROADWORK']
wdata2 = wdata2[(wdata2['DURATION_IN_HOURS'] <= (7*24)) & (wdata2['DURATION_IN_HOURS'] > 0)]
wdata2.groupby('Year').size()

Year
2021    21294
2022    21712
2023    24863
2024     9976
dtype: int64

In [35]:
wdata2['ST_RT_NO'] = (wdata2['STATE_ROUTE'].str[:4]).astype(int)

wdata2 = wdata2[['EVENTID', 'DIRECTION', 'ST_RT_NO', 'BEGIN_LOCATION_LATLONG', 'END_LOCATION_LATLONG', 'BEGINNING_DATE', 'ACT_DATETIME_OPENED', 'DURATION_IN_HOURS']]
wdata2['DIRECTION'] = wdata2['DIRECTION'].str[0]
wdata2.head()

,EVENTID,DIRECTION,ST_RT_NO,BEGIN_LOCATION_LATLONG,END_LOCATION_LATLONG,BEGINNING_DATE,ACT_DATETIME_OPENED,DURATION_IN_HOURS
0,570322,B,2080,"39.7212324915759,-76.4148035083266","39.7212224130221,-76.3964464770387",2021-01-06 07:46:16,01/06/2021 14:23:00,6.612222
151,571293,W,376,"40.4319959343462,-79.9594417422336","40.4345714299781,-79.9653194030498",2021-01-02 08:32:03,01/02/2021 11:51:00,3.315833
164,571306,N,476,"40.0664399416518,-75.3210564972281","40.0953717941557,-75.3033130917345",2021-01-02 12:28:36,01/02/2021 13:54:00,1.423333
311,571453,W,3002,"40.3108069600006,-75.1287759369276","40.3099915397151,-75.1304857375297",2021-01-03 21:51:54,01/04/2021 05:03:00,7.185000
333,571476,S,119,"40.2431835180538,-79.5766592049683","40.2402651996861,-79.5787191971731",2021-01-04 06:11:19,01/08/2021 16:48:00,106.611389


In [36]:
wdata2.to_csv('cleaned_2021-2024_workzones.csv')